Tasks

1.   Convert the height and weight columns to numerical forms
2.   Remove the unnecessary newline characters from all columns that have them.
3.   Based on the 'Joined' column, check which players have been playing at a club for more than 10 years!
4.   'Value', 'Wage' and "Release Clause' are string columns. Convert them to numbers. For eg, "M" in value column is Million, so multiply the row values by 1,000,000, etc.
5.   Some columns have 'star' characters. Strip those columns of these stars and make the columns numerical
6.   Which players are highly valuable but still underpaid (on low wages)?

In [ ]:
import numpy as np
import pandas as pd
import decimal
import os

In [ ]:
df_fifa = pd.read_csv('https://raw.githubusercontent.com/ysasamson/FIFA21/main/fifa21%20raw%20data.csv')

<ipython-input-62-3c80e1fd26b0>:1: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fifa = pd.read_csv('https://raw.githubusercontent.com/ysasamson/FIFA21/main/fifa21%20raw%20data.csv')


In [ ]:
print(df_fifa.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979 non-null  object
 17  Joined      

In [ ]:
print(df_fifa.head())

       ID               Name                      LongName  \
0  158023           L. Messi                  Lionel Messi   
1   20801  Cristiano Ronaldo  C. Ronaldo dos Santos Aveiro   
2  200389           J. Oblak                     Jan Oblak   
3  192985       K. De Bruyne               Kevin De Bruyne   
4  190871          Neymar Jr    Neymar da Silva Santos Jr.   

                                           photoUrl  \
0  https://cdn.sofifa.com/players/158/023/21_60.png   
1  https://cdn.sofifa.com/players/020/801/21_60.png   
2  https://cdn.sofifa.com/players/200/389/21_60.png   
3  https://cdn.sofifa.com/players/192/985/21_60.png   
4  https://cdn.sofifa.com/players/190/871/21_60.png   

                                           playerUrl Nationality  Age  ↓OVA  \
0  http://sofifa.com/player/158023/lionel-messi/2...   Argentina   33    93   
1  http://sofifa.com/player/20801/c-ronaldo-dos-s...    Portugal   35    92   
2  http://sofifa.com/player/200389/jan-oblak/210006/    Slo

# Preprocessing

## 1.1 Converting Height and Weight into numerical forms

In [ ]:
# Checking what was inputted in the Height and Weight column
df_fifa['Height'].unique()

array(['170cm', '187cm', '188cm', '181cm', '175cm', '184cm', '191cm',
       '178cm', '193cm', '185cm', '199cm', '173cm', '168cm', '176cm',
       '177cm', '183cm', '180cm', '189cm', '179cm', '195cm', '172cm',
       '182cm', '186cm', '192cm', '165cm', '194cm', '167cm', '196cm',
       '163cm', '190cm', '174cm', '169cm', '171cm', '197cm', '200cm',
       '166cm', '6\'2"', '164cm', '198cm', '6\'3"', '6\'5"', '5\'11"',
       '6\'4"', '6\'1"', '6\'0"', '5\'10"', '5\'9"', '5\'6"', '5\'7"',
       '5\'4"', '201cm', '158cm', '162cm', '161cm', '160cm', '203cm',
       '157cm', '156cm', '202cm', '159cm', '206cm', '155cm'], dtype=object)

In [ ]:
df_fifa['Weight'].unique()

array(['72kg', '83kg', '87kg', '70kg', '68kg', '80kg', '71kg', '91kg',
       '73kg', '85kg', '92kg', '69kg', '84kg', '96kg', '81kg', '82kg',
       '75kg', '86kg', '89kg', '74kg', '76kg', '64kg', '78kg', '90kg',
       '66kg', '60kg', '94kg', '79kg', '67kg', '65kg', '59kg', '61kg',
       '93kg', '88kg', '97kg', '77kg', '62kg', '63kg', '95kg', '100kg',
       '58kg', '183lbs', '179lbs', '172lbs', '196lbs', '176lbs', '185lbs',
       '170lbs', '203lbs', '168lbs', '161lbs', '146lbs', '130lbs',
       '190lbs', '174lbs', '148lbs', '165lbs', '159lbs', '192lbs',
       '181lbs', '139lbs', '154lbs', '157lbs', '163lbs', '98kg', '103kg',
       '99kg', '102kg', '56kg', '101kg', '57kg', '55kg', '104kg', '107kg',
       '110kg', '53kg', '50kg', '54kg', '52kg'], dtype=object)

In [ ]:
def height_to_cm(x):
  if 'cm' in x:
    return int(x.replace('cm',''))
  else:
    height = x.split("'")
    feet = int(height[0])*30.48
    inch = int(height[1][:-1])*2.54
    return round(feet+inch)

In [ ]:
df_fifa['Height'] = df_fifa['Height'].apply(height_to_cm)

In [ ]:
def weight_to_kg(x):
  if 'kg' in x:
    return int(x[:-2])
  else:
    lbs=int(x[:-3])*0.45359237
    return round(lbs)

In [ ]:
df_fifa["Weight"] = df_fifa['Weight'].apply(weight_to_kg)

In [ ]:
df_fifa.rename(columns = {'Height':'Height_cm', 'Weight':'Weight_kg'}, inplace = True)

## 1.2 Splitting Join date into different columns (year, month, day)

In [ ]:
#Convert Joined column to datetime
df_fifa['Joined'] = pd.to_datetime(df_fifa.Joined)

#Separate the year, month, date in the joined column
df_fifa['Joined_Year'] = df_fifa.Joined.dt.year
df_fifa['Joined_month'] = df_fifa.Joined.dt.month
df_fifa['Joined_day'] = df_fifa.Joined.dt.day

## 1.3 Convert the value, wage and release clause columns into integers

In [ ]:
df_fifa['Value'].unique()

array(['€103.5M', '€63M', '€120M', '€129M', '€132M', '€111M', '€120.5M',
       '€102M', '€185.5M', '€110M', '€113M', '€90.5M', '€82M', '€17.5M',
       '€83.5M', '€33.5M', '€114.5M', '€78M', '€103M', '€109M', '€92M',
       '€10M', '€76.5M', '€89.5M', '€87.5M', '€79.5M', '€124M', '€114M',
       '€95M', '€92.5M', '€105.5M', '€88.5M', '€85M', '€81.5M', '€26M',
       '€21M', '€56M', '€67.5M', '€53M', '€36.5M', '€51M', '€65.5M',
       '€46.5M', '€61.5M', '€72.5M', '€77.5M', '€43.5M', '€32.5M', '€36M',
       '€32M', '€54M', '€49.5M', '€57M', '€66.5M', '€74.5M', '€71.5M',
       '€121M', '€99M', '€67M', '€86.5M', '€93.5M', '€70M', '€62M',
       '€66M', '€58M', '€44M', '€81M', '€37M', '€14.5M', '€46M', '€47.5M',
       '€52.5M', '€54.5M', '€34.5M', '€57.5M', '€51.5M', '€44.5M', '€55M',
       '€48M', '€60.5M', '€63.5M', '€61M', '€29M', '€58.5M', '€55.5M',
       '€42M', '€40.5M', '€43M', '€45.5M', '€34M', '€26.5M', '€42.5M',
       '€35.5M', '€45M', '€41.5M', '€40M', '€11M', '€13.5M', '

In [ ]:
df_fifa['Wage'].unique()

array(['€560K', '€220K', '€125K', '€370K', '€270K', '€240K', '€250K',
       '€160K', '€260K', '€210K', '€310K', '€130K', '€350K', '€300K',
       '€190K', '€145K', '€195K', '€100K', '€140K', '€290K', '€82K',
       '€110K', '€230K', '€155K', '€200K', '€165K', '€95K', '€170K',
       '€105K', '€115K', '€150K', '€135K', '€55K', '€58K', '€81K', '€34K',
       '€120K', '€59K', '€90K', '€65K', '€56K', '€71K', '€18K', '€75K',
       '€47K', '€20K', '€84K', '€86K', '€74K', '€78K', '€27K', '€68K',
       '€85K', '€25K', '€46K', '€83K', '€54K', '€79K', '€175K', '€43K',
       '€49K', '€45K', '€38K', '€41K', '€39K', '€23K', '€51K', '€50K',
       '€87K', '€30K', '€14K', '€69K', '€31K', '€64K', '€53K', '€35K',
       '€21K', '€28K', '€17K', '€33K', '€70K', '€32K', '€89K', '€26K',
       '€40K', '€76K', '€72K', '€48K', '€36K', '€29K', '€60K', '€16K',
       '€37K', '€24K', '€52K', '€0', '€62K', '€73K', '€63K', '€19K',
       '€1K', '€66K', '€80K', '€12K', '€2K', '€42K', '€13K', '€900',
       '€5

In [ ]:
df_fifa['Release Clause'].unique()

array(['€138.4M', '€75.9M', '€159.4M', ..., '€59K', '€35K', '€64K'],
      dtype=object)

In [ ]:
def convert_to_full_int(x):
  x = x.replace('€', '')
  if 'M' in x:
    return int(float(x[:-1])*1000*1000)
  elif 'K' in x:
    return int(float(x[:-1])*1000)
  else:
    return round(float(x))

In [ ]:
df_fifa['Value'] = df_fifa['Value'].apply(convert_to_full_int)
df_fifa['Wage'] = df_fifa['Wage'].apply(convert_to_full_int)
df_fifa['Release Clause'] = df_fifa['Release Clause'].apply(convert_to_full_int)

## 1.4 Remove star symbol in columns that contain it

In [ ]:
# Determining which columns contain the star symbol
df_fifa.columns[df_fifa.astype('str').apply(lambda col: col.str.contains('★').any())]

Index(['W/F', 'SM', 'IR'], dtype='object')

In [ ]:
# Remove star and whitespace
df_fifa['IR'] = df_fifa.IR.str.strip('★').str.strip().astype(int)
df_fifa['SM'] = df_fifa.SM.str.strip('★').str.strip().astype(int)
df_fifa['W/F'] = df_fifa['W/F'].str.strip('★').str.strip().astype(int)

## 1.5 Removing unnecessary newline characters in columns that contain it

In [ ]:
# Check which column that contians newline characters.
df_fifa.columns[df_fifa.astype('str').apply(lambda col: col.str.contains('\\n').any())]

Index(['Club'], dtype='object')

In [ ]:
df_fifa['Club'] = df_fifa.Club.str.replace('\n', '', regex=False)

## 1.6 Based on the 'Joined' column, check which players have been playing at a club for more than 10 years

In [ ]:
df_fifa['Membership_Years'] = 2023-df_fifa.Joined_Year

In [ ]:
df_fifa.head()

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,SHO,PAS,DRI,DEF,PHY,Hits,Joined_Year,Joined_month,Joined_day,Membership_Years
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,FC Barcelona,...,92,91,95,38,65,771,2004,7,1,19
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,Juventus,...,93,81,89,35,77,562,2018,7,10,5
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,Atlético Madrid,...,92,78,90,52,90,150,2014,7,16,9
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,Manchester City,...,86,93,88,64,78,207,2015,8,30,8
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,Paris Saint-Germain,...,85,86,94,36,59,595,2017,8,3,6
